<a href="https://colab.research.google.com/github/aderdouri/CQFDev/blob/master/Module_01/binomial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step-by-Step Guide to Pricing Options Using the Binomial Model

### Introduction
The binomial model is a discrete-time method for pricing options. It simulates the possible movements of the underlying asset price over multiple time steps and calculates the option price through backward induction.

### Step 1: Define the Model Parameters}
The required parameters for the binomial tree are:
- Current stock price: $S_0$
- Strike price: $K$
- Risk-free interest rate: $r$
- Volatility: $\sigma$
- Time to maturity: $T$
- Number of time steps: $N$

The time step size is computed as:
\begin{equation}
    \Delta t = \frac{T}{N}
\end{equation}

### Step 2: Construct the Binomial Price Tree
The up ($u$) and down ($d$) factors are defined as:
\begin{equation}
    u = e^{\sigma \sqrt{\Delta t}}
\end{equation}
\begin{equation}
    d = \frac{1}{u} = e^{-\sigma \sqrt{\Delta t}}
\end{equation}
The risk-neutral probability is given by:
\begin{equation}
    p = \frac{e^{r \Delta t} - d}{u - d}
\end{equation}
The stock price at each node $(i, j)$ is computed as:
\begin{equation}
    S_{i,j} = S_0 \cdot u^j \cdot d^{i-j}
\end{equation}

### Step 3: Compute Option Payoff at Maturity
At expiration ($t = T$), the option value is determined by its intrinsic value:
- European call option:
\begin{equation}
    V_N^j = \max(S_{N,j} - K, 0)
\end{equation}
- European put option:
\begin{equation}
    V_N^j = \max(K - S_{N,j}, 0)
\end{equation}

### Step 4: Backward Induction for Option Pricing
We work backward from the terminal nodes to the initial node. The option price at each node is computed as:
\begin{equation}
    V_i^j = e^{-r \Delta t} \cdot \left( p \cdot V_{i+1}^{j+1} + (1 - p) \cdot V_{i+1}^{j} \right)
\end{equation}
For American options, the value at each node is the maximum of the intrinsic value and the continuation value:
\begin{equation}
    V_i^j = \max(\text{Intrinsic Value}, \text{Continuation Value})
\end{equation}

### Step 5: Extract the Option Price
The option price at time $t = 0$ is the value at the root node:
\begin{equation}
    V_0^0
\end{equation}
This represents the fair value of the option under the binomial model.

## Conclusion
The binomial model provides a simple yet powerful way to price options, allowing for flexibility in handling American options and accommodating different market conditions.

In [1]:
# prompt: implement the binomial model for option pricing

import math

def binomial_option_pricing(S0, K, r, sigma, T, N, option_type='call'):
    """
    Prices European and American options using the binomial model.

    Args:
        S0: Current stock price.
        K: Strike price.
        r: Risk-free interest rate.
        sigma: Volatility.
        T: Time to maturity.
        N: Number of time steps.
        option_type: 'call' or 'put'.

    Returns:
        The option price at time t=0.
    """

    dt = T / N
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u
    p = (math.exp(r * dt) - d) / (u - d)

    # Initialize option values at maturity
    option_values = [0] * (N + 1)
    for j in range(N + 1):
        S_T = S0 * (u ** j) * (d ** (N - j))
        if option_type == 'call':
            option_values[j] = max(0, S_T - K)
        elif option_type == 'put':
            option_values[j] = max(0, K - S_T)

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            S_i = S0 * (u ** j) * (d ** (i - j))
            continuation_value = math.exp(-r * dt) * (p * option_values[j + 1] + (1 - p) * option_values[j])

            if option_type == 'call':
                intrinsic_value = max(0, S_i - K)
                option_values[j] = max(intrinsic_value, continuation_value)  # American option
            elif option_type == 'put':
                intrinsic_value = max(0, K-S_i)
                option_values[j] = max(intrinsic_value,continuation_value) #American Option
    return option_values[0]


# Example usage:
S0 = 100  # Current stock price
K = 105   # Strike price
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
T = 1     # Time to maturity (in years)
N = 100   # Number of time steps

call_price = binomial_option_pricing(S0, K, r, sigma, T, N, option_type='call')
put_price = binomial_option_pricing(S0, K, r, sigma, T, N, option_type='put')

print(f"Call option price: {call_price:.4f}")
print(f"Put option price: {put_price:.4f}")

Call option price: 8.0262
Put option price: 8.7476
